In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from category_encoders import BinaryEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score


font_path = '../Font/GmarketSansTTFMedium.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

plt.rcParams['axes.unicode_minus'] = False

In [43]:
seoul = pd.read_csv('../Data/preprocessing/seoul_real_transcation_price.csv')

In [44]:
seoul.head(5)

,시도명,시군구명,관리기관,회사명,공장구분,단지명,설립구분,입주형태,보유구분,등록구분,...,지식산업센터명,대표업종,업종명,업종코드,차수,법인주소,필지수,공장주소,공장주소_지번,공장관리번호
0,서울특별시,종로구,서울특별시 종로구,남일문화 (주),개별,,일반,해당없음,임대,신규등록,...,,18111,경 인쇄업 외 2 종,"181,111,811,318,119",10,서울특별시 종로구 자하문로16길 8 (창성동),1,서울특별시 종로구 자하문로16길 8 (창성동),서울특별시 종로구 창성동 87-1번지,1.110000e+14
1,서울특별시,종로구,서울특별시 종로구,(주)에취.알.디,개별,,일반,해당없음,임대,등록변경,...,,33932,전시용 모형 제조업,33932,10,서울특별시 종로구 통의동 35-69번지,1,서울특별시 종로구 통의동 35-69번지,서울특별시 종로구 통의동 35-69번지,1.110000e+14
2,서울특별시,종로구,서울특별시 종로구,(주)코리아쉬핑가제트,개별,,일반,해당없음,자가,등록변경,...,,58113,일반 서적 출판업 외 3 종,"58,113,181,111,811,300,000",10,서울특별시 도봉구 창동 181-17번지 쌍용아파트 115-1902,1,서울특별시 종로구 자하문로2길 13-3 (통의동),서울특별시 종로구 통의동 35-6번지,1.110000e+14
3,서울특별시,종로구,서울특별시 종로구,삼영DP,개별,,일반,해당없음,임대,신규등록,...,,18111,경 인쇄업 외 2 종,"181,111,811,318,119",10,서울특별시 종로구 사직로 125 (적선동),1,서울특별시 종로구 사직로 125 (적선동),서울특별시 종로구 적선동 107-1번지,1.110000e+14
4,서울특별시,종로구,서울특별시 종로구,삼진기획,개별,,일반,해당없음,임대,등록변경,...,,18119,기타 인쇄업 외 1 종,"1,811,918,113",10,서울특별시 서대문구 홍제동 -번지 문화촌현대아파트 103-1101호,1,서울특별시 종로구 적선동 2번지,서울특별시 종로구 적선동 2번지,1.110000e+14


In [45]:
seoul.columns

Index(['시도명', '시군구명', '관리기관', '회사명', '공장구분', '단지명', '설립구분', '입주형태', '보유구분',
       '등록구분', '전화번호', '남자종업원', '여자종업원', '외국인남자종업원', '외국인여자종업원', '종업원합계',
       '생산품', '원자재', '공장규모', '용도지역', '지목', '용지면적', '제조시설면적', '부대시설면적', '건축면적',
       '지식산업센터명', '대표업종', '업종명', '업종코드', '차수', '법인주소', '필지수', '공장주소',
       '공장주소_지번', '공장관리번호'],
      dtype='object')

### 삭제할 칼럼
시도명, 시군구명, 관리기관, 회사명, 단지명, 전화번호, 법인주소, 필지수, 공장주소, 공장관리번호, 시

### 보류 칼럼 (일단 삭제)
생산품, 원자재, 업종명, 대표업종, 차수, 필지수

### 전처리 진행
용도지역, 업종코드 값 000 제거, 공장주소 지번 -> 동으로 쪼개기

In [46]:
seoul = seoul.drop(columns=['시도명', '시군구명', '관리기관', '회사명', '단지명', '전화번호', '법인주소', '필지수', '공장주소', '공장관리번호']) #1차
seoul = seoul.drop(columns=['생산품', '원자재', '업종명', '업종코드', '차수']) #2차

## 시각화 함수 

### 공장주소_지번을 시, 구, 동으로 구분하기

In [47]:
address = seoul['공장주소_지번'].str.split(n=2, expand=True)
address[2] = address[2].str.split().str[0] #남은 부분에서 첫번째만 분리

print(address)

address.columns = ['시', '구', '동']
seoul = pd.concat([seoul, address[['시', '구', '동']]], axis=1)
seoul = seoul.drop(columns=['공장주소_지번', '시', '동'])


           0    1    2
0      서울특별시  종로구  창성동
1      서울특별시  종로구  통의동
2      서울특별시  종로구  통의동
3      서울특별시  종로구  적선동
4      서울특별시  종로구  적선동
...      ...  ...  ...
11628  서울특별시  강동구  천호동
11629  서울특별시  강동구  천호동
11630  서울특별시  강동구  천호동
11631  서울특별시  강동구  천호동
11632  서울특별시  강동구  천호동

[11633 rows x 3 columns]


In [48]:
object_columns = ['공장구분', '설립구분', '입주형태', '보유구분', '등록구분', '공장규모', '용도지역', '지목', '지식산업센터명', '대표업종', '구']
int_columns = ['남자종업원', '여자종업원', '외국인남자종업원', '외국인여자종업원', '종업원합계','용지면적', '제조시설면적', '부대시설면적', '건축면적']

In [49]:
# object 타입 변환
for column in object_columns:
    seoul[column] = seoul[column].astype('object')

for column in int_columns:
    seoul[column] = seoul[column].astype('int')

In [50]:
seoul['입주형태'] = seoul['입주형태'].str.strip() # 공백제거
seoul['입주형태'].replace('', '해당없음', inplace=True)
seoul['입주형태'].fillna('해당없음', inplace=True)

In [ ]:
seoul['용도지역'] = seoul['용도지역'].str.strip() # 공백제거
seoul['용도지역'].replace('', '해당없음', inplace=True)
seoul['용도지역'].fillna('해당없음', inplace=True)

In [51]:
seoul['지목'] = seoul['지목'].str.strip() # 공백제거
seoul['지목'].replace('', '해당없음', inplace=True)
seoul['지목'].fillna('해당없음', inplace=True)

In [52]:
seoul['지식산업센터명'] = seoul['지식산업센터명'].str.strip() # 공백제거
seoul['지식산업센터명'].replace('', '해당없음', inplace=True)
seoul['지식산업센터명'].fillna('해당없음', inplace=True)

In [53]:
seoul.isnull().sum()

공장구분        0
설립구분        0
입주형태        0
보유구분        0
등록구분        0
남자종업원       0
여자종업원       0
외국인남자종업원    0
외국인여자종업원    0
종업원합계       0
공장규모        0
용도지역        0
지목          0
용지면적        0
제조시설면적      0
부대시설면적      0
건축면적        0
지식산업센터명     0
대표업종        0
구           0
dtype: int64

In [54]:
seoul['구'].value_counts()

금천구     3455
성동구     1357
구로구     1215
중구      1190
영등포구     845
강서구      642
송파구      428
중랑구      341
동대문구     265
강남구      228
서초구      208
광진구      174
종로구      166
도봉구      155
강동구      129
양천구      108
노원구      105
성북구      103
마포구      102
은평구       94
강북구       89
용산구       77
관악구       65
서대문구      52
동작구       40
Name: 구, dtype: int64

In [55]:
label_encoders = {}
for column in seoul.select_dtypes(include=['object']).columns:
    if column != '구':
        label_encoders[column] = LabelEncoder()
        seoul[column] = label_encoders[column].fit_transform(seoul[column])

# '구' 칼럼 라벨 인코딩
label_encoder_gu = LabelEncoder()
seoul['구'] = label_encoder_gu.fit_transform(seoul['구'])


In [56]:
# seoul.to_csv('./model_file.csv', index = False)

In [57]:
X = seoul.drop(columns=['구'])
y = seoul['구']

In [58]:
print(X.shape, y.shape)

(11633, 19) (11633,)


In [59]:
X

,공장구분,설립구분,입주형태,보유구분,등록구분,남자종업원,여자종업원,외국인남자종업원,외국인여자종업원,종업원합계,공장규모,용도지역,지목,용지면적,제조시설면적,부대시설면적,건축면적,지식산업센터명,대표업종
0,0,1,8,0,2,0,0,0,0,0,1,57,2,238,57,69,126,223,105
1,0,1,8,0,0,5,2,0,0,7,1,78,2,157,79,78,157,223,396
2,0,1,8,1,0,12,12,0,0,24,1,78,2,160,92,67,160,223,422
3,0,1,8,0,2,0,0,0,0,0,1,59,2,280,35,88,123,223,105
4,0,1,8,0,0,7,2,0,0,9,1,34,2,122,159,18,177,223,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11628,0,1,5,0,0,1,4,0,0,5,1,58,2,104,104,0,104,223,68
11629,0,1,8,1,0,0,0,0,0,0,1,78,2,164,110,53,164,223,384
11630,0,1,5,0,0,7,0,0,0,7,1,58,2,125,114,11,125,223,77
11631,0,1,5,0,2,2,3,0,0,5,1,74,2,104,104,0,104,223,42


In [60]:
y.value_counts()

7     3455
15    1357
6     1215
23    1190
19     845
3      642
17     428
24     341
10     265
0      228
14     208
5      174
22     166
9      155
1      129
18     108
8      105
16     103
12     102
21      94
2       89
20      77
4       65
13      52
11      40
Name: 구, dtype: int64

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(7910, 19) (1978, 19) (1745, 19) (7910,) (1978,) (1745,)


In [62]:
## label encoding
from sklearn.preprocessing import LabelEncoder
categorical_features = list(X_train.dtypes[X_train.dtypes == 'object'.index])

# 라벨 인코딩 호기화
le = LabelEncoder()

# Label encoding 적용
for feature in categorical_features:
    X_train[feature] = le.fit_transform(X_train[feature])
    X_val[feature] = le.fit_transform(X_val[feature])
    X_test[feature] = le.fit_transform(X_test[feature])

In [63]:
# from sklearn.preprocessing import LabelEncoder
# from category_encoders import TargetEncoder

# # 범주형 피처들을 찾아 리스트에 저장
# categorical_features = list(X_train.dtypes[X_train.dtypes == "object"].index)

# display(categorical_features)
# # TargetEncoder 초기화
# te = TargetEncoder()

# # Target encoding 적용
# X_train[categorical_features] = te.fit_transform(X_train[categorical_features], y_train)
# X_val[categorical_features] = te.transform(X_val[categorical_features])
# X_test[categorical_features] = te.transform(X_test[categorical_features])

# # 결과 확인
# display(X_train)
# display(X_test)

In [64]:
print(X_train.info(), X.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7910 entries, 2548 to 5416
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   공장구분      7910 non-null   int32
 1   설립구분      7910 non-null   int32
 2   입주형태      7910 non-null   int32
 3   보유구분      7910 non-null   int32
 4   등록구분      7910 non-null   int32
 5   남자종업원     7910 non-null   int32
 6   여자종업원     7910 non-null   int32
 7   외국인남자종업원  7910 non-null   int32
 8   외국인여자종업원  7910 non-null   int32
 9   종업원합계     7910 non-null   int32
 10  공장규모      7910 non-null   int32
 11  용도지역      7910 non-null   int32
 12  지목        7910 non-null   int32
 13  용지면적      7910 non-null   int32
 14  제조시설면적    7910 non-null   int32
 15  부대시설면적    7910 non-null   int32
 16  건축면적      7910 non-null   int32
 17  지식산업센터명   7910 non-null   int32
 18  대표업종      7910 non-null   int32
dtypes: int32(19)
memory usage: 648.9 KB
None 공장구분        0
설립구분        0
입주형태        0
보유구분        0
등록구분       

In [65]:
X_train

,공장구분,설립구분,입주형태,보유구분,등록구분,남자종업원,여자종업원,외국인남자종업원,외국인여자종업원,종업원합계,공장규모,용도지역,지목,용지면적,제조시설면적,부대시설면적,건축면적,지식산업센터명,대표업종
2548,0,1,8,1,2,10,1,2,0,13,1,9,2,495,245,54,299,223,165
5896,1,0,4,1,3,32,6,1,0,39,1,9,0,208,816,636,1452,96,250
2660,0,1,8,0,0,5,12,0,0,17,1,11,2,904,72,26,98,223,65
8386,1,3,4,1,3,2,9,0,0,11,1,9,0,66,271,175,446,172,62
3536,0,1,5,0,2,4,0,0,0,4,1,58,2,0,131,160,292,223,268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,0,1,5,0,0,1,2,0,0,3,1,1,2,170,22,148,170,223,39
9825,0,1,8,0,3,24,1,0,0,25,1,9,0,601,412,220,632,223,347
4383,0,1,8,1,3,11,2,0,0,13,1,9,2,359,0,110,110,223,229
1315,0,1,5,0,2,1,1,0,0,2,1,34,0,0,65,0,65,69,64


In [66]:
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import os

In [67]:
# 결과를 CSV 파일에 저장하는 콜백 함수
def save_trial_to_csv(study, trial, model_name):
    header = ['Trial Number', 'Model', 'Params', 'Value']
    file_exists = os.path.isfile('optuna_trials.csv')
    
    with open('optuna_trials.csv', mode='a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        
        params = trial.params
        writer.writerow([trial.number, model_name, params, trial.value])


In [68]:
# Optuna 목적 함수 - XGBoost
def objective_xgb(trial, X_train, y_train, X_val, y_val):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10)
    }
    model = XGBClassifier(**params, use_label_encoder=False)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='mlogloss',
              early_stopping_rounds=10,
              verbose=False)
    score = model.score(X_val, y_val)
    return score

In [69]:
# Optuna 목적 함수 - LightGBM
def objective_lgbm(trial, X_train, y_train, X_val, y_val):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_split_gain': trial.suggest_float('min_split_gain', 0, 10)
    }
    model = LGBMClassifier(**params)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='multi_logloss'
    )
    
    # Manual early stopping implementation
    early_stopping_rounds = 10
    best_iteration = len(model.evals_result_['valid_0']['multi_logloss']) - 1
    for i in range(len(model.evals_result_['valid_0']['multi_logloss']) - early_stopping_rounds):
        if model.evals_result_['valid_0']['multi_logloss'][i] < min(model.evals_result_['valid_0']['multi_logloss'][i+1:i+early_stopping_rounds+1]):
            best_iteration = i
            break
    
    model.set_params(n_estimators=best_iteration + 1)
    model.fit(X_train, y_train)  # Refit the model with the best iteration
    
    score = model.score(X_val, y_val)
    return score

In [70]:
# XGBoost 하이퍼파라미터 최적화
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(lambda trial: objective_xgb(trial, X_train, y_train, X_val, y_val), n_trials=100, callbacks=[lambda study, trial: save_trial_to_csv(study, trial, 'XGBoost')])

[I 2024-06-22 00:32:48,928] A new study created in memory with name: no-name-7059cc1a-6a2f-403b-a10c-cfce02ed0d5b
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2024-06-22 00:32:51,382] Trial 0 finished with value: 0.6107178968655207 and parameters: {'max_depth': 5, 'learning_rate': 0.14490660245371065, 'n_estimators': 249, 'subsample': 0.67416429401568, 'colsample_bytree': 0.9827896109914916, 'gamma': 5.173137443437258, 'min_child_weight': 1, 'reg_alpha': 1.4463764627801623, 'reg_lambda': 4.959607001536799}. Best is trial 0 with valu

In [71]:
# LightGBM 하이퍼파라미터 최적화
study_lgbm = optuna.create_study(direction='maximize')
study_lgbm.optimize(lambda trial: objective_lgbm(trial, X_train, y_train, X_val, y_val), n_trials=100, callbacks=[lambda study, trial: save_trial_to_csv(study, trial, 'LightGBM')])

[I 2024-06-22 00:36:22,638] A new study created in memory with name: no-name-ac9838ba-fff4-403d-9ace-ec9d8e311304


[1]	valid_0's multi_logloss: 1.95307
[2]	valid_0's multi_logloss: 8.32631
[3]	valid_0's multi_logloss: 20.959
[4]	valid_0's multi_logloss: 21.8352
[5]	valid_0's multi_logloss: 25.4924
[6]	valid_0's multi_logloss: 28.0811
[7]	valid_0's multi_logloss: 30.4443
[8]	valid_0's multi_logloss: 24.9091
[9]	valid_0's multi_logloss: 28.0411
[10]	valid_0's multi_logloss: 23.0547
[11]	valid_0's multi_logloss: 21.8728
[12]	valid_0's multi_logloss: 25.0743
[13]	valid_0's multi_logloss: 20.9811
[14]	valid_0's multi_logloss: 23.8613
[15]	valid_0's multi_logloss: 21.031
[16]	valid_0's multi_logloss: 20.1433
[17]	valid_0's multi_logloss: 19.5541
[18]	valid_0's multi_logloss: 19.5951
[19]	valid_0's multi_logloss: 18.9886
[20]	valid_0's multi_logloss: 18.6732
[21]	valid_0's multi_logloss: 18.8725
[22]	valid_0's multi_logloss: 18.2729
[23]	valid_0's multi_logloss: 18.1237
[24]	valid_0's multi_logloss: 17.7772
[25]	valid_0's multi_logloss: 17.5355
[26]	valid_0's multi_logloss: 17.3079
[27]	valid_0's multi_lo

[I 2024-06-22 00:36:24,626] Trial 0 finished with value: 0.46461071789686553 and parameters: {'num_leaves': 34, 'learning_rate': 0.903118740055009, 'n_estimators': 212, 'min_child_samples': 79, 'subsample': 0.7867413453203699, 'colsample_bytree': 0.6755800781138098, 'reg_alpha': 9.672394621642255, 'reg_lambda': 1.0920303188268388, 'min_split_gain': 9.949579891503612}. Best is trial 0 with value: 0.46461071789686553.


[1]	valid_0's multi_logloss: 2.4891
[2]	valid_0's multi_logloss: 2.4891
[3]	valid_0's multi_logloss: 2.4891
[4]	valid_0's multi_logloss: 2.4891
[5]	valid_0's multi_logloss: 2.4891
[6]	valid_0's multi_logloss: 2.4891
[7]	valid_0's multi_logloss: 2.4891
[8]	valid_0's multi_logloss: 2.4891
[9]	valid_0's multi_logloss: 2.4891
[10]	valid_0's multi_logloss: 2.4891
[11]	valid_0's multi_logloss: 2.4891
[12]	valid_0's multi_logloss: 2.4891
[13]	valid_0's multi_logloss: 2.4891
[14]	valid_0's multi_logloss: 2.4891
[15]	valid_0's multi_logloss: 2.4891
[16]	valid_0's multi_logloss: 2.4891
[17]	valid_0's multi_logloss: 2.4891
[18]	valid_0's multi_logloss: 2.4891
[19]	valid_0's multi_logloss: 2.4891
[20]	valid_0's multi_logloss: 2.4891
[21]	valid_0's multi_logloss: 2.4891
[22]	valid_0's multi_logloss: 2.4891
[23]	valid_0's multi_logloss: 2.4891
[24]	valid_0's multi_logloss: 2.4891
[25]	valid_0's multi_logloss: 2.4891
[26]	valid_0's multi_logloss: 2.4891
[27]	valid_0's multi_logloss: 2.4891
[28]	valid

[I 2024-06-22 00:36:31,431] Trial 1 finished with value: 0.29575328614762386 and parameters: {'num_leaves': 36, 'learning_rate': 1.1252622485286323e-08, 'n_estimators': 163, 'min_child_samples': 8, 'subsample': 0.9270194140974914, 'colsample_bytree': 0.5849163683353349, 'reg_alpha': 6.5176908916592735, 'reg_lambda': 5.664667115545513, 'min_split_gain': 2.043006193277291}. Best is trial 0 with value: 0.46461071789686553.


...

[I 2024-06-22 00:44:30,221] Trial 97 finished with value: 0.6718907987866531 and parameters: {'num_leaves': 250, 'learning_rate': 0.02155732276740629, 'n_estimators': 281, 'min_child_samples': 52, 'subsample': 0.7686432836950158, 'colsample_bytree': 0.7895505831046785, 'reg_alpha': 1.172247157878829, 'reg_lambda': 7.156268016248362, 'min_split_gain': 0.5548317519631287}. Best is trial 74 with value: 0.6830131445904954.


[1]	valid_0's multi_logloss: 2.32226
[2]	valid_0's multi_logloss: 2.19058
[3]	valid_0's multi_logloss: 2.07844
[4]	valid_0's multi_logloss: 1.98493
[5]	valid_0's multi_logloss: 1.90391
[6]	valid_0's multi_logloss: 1.8322
[7]	valid_0's multi_logloss: 1.77111
[8]	valid_0's multi_logloss: 1.71722
[9]	valid_0's multi_logloss: 1.67234
[10]	valid_0's multi_logloss: 1.63547
[11]	valid_0's multi_logloss: 1.59668
[12]	valid_0's multi_logloss: 1.56398
[13]	valid_0's multi_logloss: 1.53416
[14]	valid_0's multi_logloss: 1.50714
[15]	valid_0's multi_logloss: 1.4828
[16]	valid_0's multi_logloss: 1.46124
[17]	valid_0's multi_logloss: 1.43743
[18]	valid_0's multi_logloss: 1.41976
[19]	valid_0's multi_logloss: 1.4022
[20]	valid_0's multi_logloss: 1.38456
[21]	valid_0's multi_logloss: 1.37052
[22]	valid_0's multi_logloss: 1.356
[23]	valid_0's multi_logloss: 1.34361
[24]	valid_0's multi_logloss: 1.33104
[25]	valid_0's multi_logloss: 1.32135
[26]	valid_0's multi_logloss: 1.3107
[27]	valid_0's multi_loglos

[I 2024-06-22 00:44:34,280] Trial 98 finished with value: 0.6723963599595552 and parameters: {'num_leaves': 239, 'learning_rate': 0.07551561435430691, 'n_estimators': 267, 'min_child_samples': 37, 'subsample': 0.733073083012917, 'colsample_bytree': 0.7475887098981671, 'reg_alpha': 0.7233642773625212, 'reg_lambda': 8.320560689863784, 'min_split_gain': 0.9730108092972418}. Best is trial 74 with value: 0.6830131445904954.


[1]	valid_0's multi_logloss: 2.15984
[2]	valid_0's multi_logloss: 1.95432
[3]	valid_0's multi_logloss: 1.80935
[4]	valid_0's multi_logloss: 1.69384
[5]	valid_0's multi_logloss: 1.60666
[6]	valid_0's multi_logloss: 1.54163
[7]	valid_0's multi_logloss: 1.48315
[8]	valid_0's multi_logloss: 1.43738
[9]	valid_0's multi_logloss: 1.40157
[10]	valid_0's multi_logloss: 1.37515
[11]	valid_0's multi_logloss: 1.34609
[12]	valid_0's multi_logloss: 1.32587
[13]	valid_0's multi_logloss: 1.30582
[14]	valid_0's multi_logloss: 1.29042
[15]	valid_0's multi_logloss: 1.27845
[16]	valid_0's multi_logloss: 1.2677
[17]	valid_0's multi_logloss: 1.25392
[18]	valid_0's multi_logloss: 1.24701
[19]	valid_0's multi_logloss: 1.24015
[20]	valid_0's multi_logloss: 1.23071
[21]	valid_0's multi_logloss: 1.22518
[22]	valid_0's multi_logloss: 1.22062
[23]	valid_0's multi_logloss: 1.21655
[24]	valid_0's multi_logloss: 1.2127
[25]	valid_0's multi_logloss: 1.21006
[26]	valid_0's multi_logloss: 1.20686
[27]	valid_0's multi_lo

[I 2024-06-22 00:44:36,958] Trial 99 finished with value: 0.6547017189079879 and parameters: {'num_leaves': 232, 'learning_rate': 0.14759138124088575, 'n_estimators': 295, 'min_child_samples': 35, 'subsample': 0.8072584754891492, 'colsample_bytree': 0.7667668275099725, 'reg_alpha': 1.3852771680018274, 'reg_lambda': 6.532095688420844, 'min_split_gain': 2.0899153796074192}. Best is trial 74 with value: 0.6830131445904954.


In [72]:
# 최적의 하이퍼파라미터 출력
best_params_xgb = study_xgb.best_trial.params
best_value_xgb = study_xgb.best_trial.value
best_params_lgbm = study_lgbm.best_trial.params
best_value_lgbm = study_lgbm.best_trial.value

print("Best trial parameters for XGBoost:", best_params_xgb)
print("Best score for XGBoost:", best_value_xgb)
print("Best trial parameters for LightGBM:", best_params_lgbm)
print("Best score for LightGBM:", best_value_lgbm)


Best trial parameters for XGBoost: {'max_depth': 5, 'learning_rate': 0.1300918726059749, 'n_estimators': 220, 'subsample': 0.8604752773818702, 'colsample_bytree': 0.947536499090015, 'gamma': 0.28310148651580475, 'min_child_weight': 2, 'reg_alpha': 0.04775304383167353, 'reg_lambda': 3.1796604958721355}
Best score for XGBoost: 0.673407482305359
Best trial parameters for LightGBM: {'num_leaves': 183, 'learning_rate': 0.05301025347665236, 'n_estimators': 267, 'min_child_samples': 49, 'subsample': 0.8280708153469658, 'colsample_bytree': 0.8115115502834236, 'reg_alpha': 0.24383119653567353, 'reg_lambda': 6.961964813669102, 'min_split_gain': 0.6047767141120094}
Best score for LightGBM: 0.6830131445904954


In [73]:
# 최적의 하이퍼파라미터로 모델 훈련
xgb_model = XGBClassifier(**best_params_xgb, use_label_encoder=False)
lgbm_model = LGBMClassifier(**best_params_lgbm)

In [74]:
# 모델 학습
xgb_model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='mlogloss',
              early_stopping_rounds=10,
              verbose=False)

lgbm_model.fit(X_train, y_train,
               eval_set=[(X_val, y_val)],
               eval_metric='multi_logloss')


c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\vivid\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	valid_0's multi_logloss: 2.36479
[2]	valid_0's multi_logloss: 2.25988
[3]	valid_0's multi_logloss: 2.16775
[4]	valid_0's multi_logloss: 2.08974
[5]	valid_0's multi_logloss: 2.01722
[6]	valid_0's multi_logloss: 1.95506
[7]	valid_0's multi_logloss: 1.89785
[8]	valid_0's multi_logloss: 1.84616
[9]	valid_0's multi_logloss: 1.80207
[10]	valid_0's multi_logloss: 1.76531
[11]	valid_0's multi_logloss: 1.72464
[12]	valid_0's multi_logloss: 1.69285
[13]	valid_0's multi_logloss: 1.66156
[14]	valid_0's multi_logloss: 1.632
[15]	valid_0's multi_logloss: 1.60477
[16]	valid_0's multi_logloss: 1.58021
[17]	valid_0's multi_logloss: 1.5548
[18]	valid_0's multi_logloss: 1.53491
[19]	valid_0's multi_logloss: 1.51316
[20]	valid_0's multi_logloss: 1.49233
[21]	valid_0's multi_logloss: 1.47362
[22]	valid_0's multi_logloss: 1.45612
[23]	valid_0's multi_logloss: 1.4391
[24]	valid_0's multi_logloss: 1.42516
[25]	valid_0's multi_logloss: 1.41155
[26]	valid_0's multi_logloss: 1.39769
[27]	valid_0's multi_logl

LGBMClassifier(colsample_bytree=0.8115115502834236,
               learning_rate=0.05301025347665236, min_child_samples=49,
               min_split_gain=0.6047767141120094, n_estimators=267,
               num_leaves=183, reg_alpha=0.24383119653567353,
               reg_lambda=6.961964813669102, subsample=0.8280708153469658)

In [75]:
# Voting Ensemble
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[
    ('xgb', xgb_model), ('lgbm', lgbm_model)], voting='soft')

# Voting 모델 학습
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.947536499090015,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=0.28310148651580475,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_...
                                            n_estimators=220, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='multi:softprob', ...)),
                             ('lgbm',
                              LGBMClassifier(colsample_bytree=0.8115115502834236,
                                             learning_rate=0.05301025347665236,
                                             min_child_samples=49,
                                             min_split_gain=0.6047767141120094,
                                             n_estimators=267, num_leaves=183,
                                             reg_alpha=0.24383119653567353,
                                             reg_lambda=6.961964813669102,
                                             subsample=0.8280708153469658))],
                 voting='soft')

In [76]:
# 테스트 데이터로 모델 평가
y_pred = voting_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Voting Ensemble Test Accuracy: {test_accuracy:.4f}")


Voting Ensemble Test Accuracy: 0.6819


In [77]:

# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(voting_clf, X, y, cv=cv, scoring='accuracy')
print(f"Voting Ensemble Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")


Voting Ensemble Cross-Validation Accuracy: 0.6966 ± 0.0074


In [78]:
# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(voting_clf, X, y, cv=cv, scoring='accuracy')
print(f"Voting Ensemble Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")


Voting Ensemble Cross-Validation Accuracy: 0.7013 ± 0.0086


##  xgboost, lgbm voting 후 cross validation 15회가 성능이 가장 좋음

In [79]:
# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)
scores = cross_val_score(voting_clf, X, y, cv=cv, scoring='accuracy')
print(f"Voting Ensemble Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

Voting Ensemble Cross-Validation Accuracy: 0.7033 ± 0.0100


In [80]:
# 교차 검증으로 모델 평가
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
scores = cross_val_score(voting_clf, X, y, cv=cv, scoring='accuracy')
print(f"Voting Ensemble Cross-Validation Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

Voting Ensemble Cross-Validation Accuracy: 0.7027 ± 0.0178
